<a href="https://colab.research.google.com/github/tabaahi07/Research_Paper/blob/main/Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==================== INSTALL DEPENDENCIES ====================
!pip install scikit-image tabulate

# ==================== IMPORTS ====================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from skimage.feature import hog
from tabulate import tabulate

import warnings
warnings.filterwarnings('ignore')

In [2]:
# ==================== LOAD DATA ====================
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# ==================== RESHAPE AND SCALE ====================
scaler = MinMaxScaler()
X_train_flat = X_train.reshape(-1, 28 * 28)
X_test_flat = X_test.reshape(-1, 28 * 28)

X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# ==================== HOG FEATURE EXTRACTION ====================
def extract_hog_features(images):
    hog_features = []
    for img in images:
        features = hog(img, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), block_norm='L2-Hys')
        hog_features.append(features)
    return np.array(hog_features)

X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

In [4]:
# ==================== SVM with TUNING ====================
param_grid_svm = {'C': [1, 10], 'gamma': ['scale'], 'kernel': ['rbf']}
svm = GridSearchCV(SVC(), param_grid_svm, cv=3, verbose=0)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)

In [5]:
# ==================== SVM + HOG ====================
svm_hog = GridSearchCV(SVC(), param_grid_svm, cv=3, verbose=0)
svm_hog.fit(X_train_hog, y_train)
y_pred_svm_hog = svm_hog.predict(X_test_hog)

In [6]:
# ==================== RANDOM FOREST ====================
param_grid_rf = {'n_estimators': [100, 200], 'max_features': ['sqrt']}
rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=3, verbose=0)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

# ==================== RF + HOG ====================
rf_hog = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=3, verbose=0)
rf_hog.fit(X_train_hog, y_train)
y_pred_rf_hog = rf_hog.predict(X_test_hog)

In [7]:
# ==================== CNN IMPROVED ====================
X_train_cnn = X_train.reshape(-1,28,28,1).astype('float32')/255
X_test_cnn = X_test.reshape(-1,28,28,1).astype('float32')/255

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
datagen.fit(X_train_cnn)

history = model.fit(datagen.flow(X_train_cnn, y_train, batch_size=64),
                    epochs=10, validation_data=(X_test_cnn, y_test), verbose=1)

y_pred_cnn = np.argmax(model.predict(X_test_cnn), axis=-1)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 92s 94ms/step - accuracy: 0.6676 - loss: 1.0520 - val_accuracy: 0.9787 - val_loss: 0.0660
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 90s 96ms/step - accuracy: 0.9164 - loss: 0.2762 - val_accuracy: 0.9867 - val_loss: 0.0391
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 89s 95ms/step - accuracy: 0.9409 - loss: 0.2035 - val_accuracy: 0.9886 - val_loss: 0.0319
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 89s 95ms/step - accuracy: 0.9508 - loss: 0.1658 - val_accuracy: 0.9896 - val_loss: 0.0375
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 88s 94ms/step - accuracy: 0.9565 - loss: 0.1502 - val_accuracy: 0.9870 - val_loss: 0.0407
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 85s 91ms/step - accuracy: 0.9595 - loss: 0.1427 - val_accuracy: 0.9907 - val_loss: 0.0272
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 87s 93ms/step - accuracy: 0.9624 - loss: 0.1289 - val_accuracy: 0.9909 - val_loss: 0.0293
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 88s 93ms/step - accuracy: 0.9645 - loss: 0.1239 - 

In [9]:
# ==================== EVALUATE MODELS ====================
results = {
    'SVM': {
        'accuracy': accuracy_score(y_test, y_pred_svm),
        'f1_score': f1_score(y_test, y_pred_svm, average='weighted'),
        'precision': precision_score(y_test, y_pred_svm, average='weighted'),
        'recall': recall_score(y_test, y_pred_svm, average='weighted')
    },
    'SVM_HOG': {
        'accuracy': accuracy_score(y_test, y_pred_svm_hog),
        'f1_score': f1_score(y_test, y_pred_svm_hog, average='weighted'),
        'precision': precision_score(y_test, y_pred_svm_hog, average='weighted'),
        'recall': recall_score(y_test, y_pred_svm_hog, average='weighted')
    },
    'Random Forest': {
        'accuracy': accuracy_score(y_test, y_pred_rf),
        'f1_score': f1_score(y_test, y_pred_rf, average='weighted'),
        'precision': precision_score(y_test, y_pred_rf, average='weighted'),
        'recall': recall_score(y_test, y_pred_rf, average='weighted')
    },
    'Random Forest_HOG': {
        'accuracy': accuracy_score(y_test, y_pred_rf_hog),
        'f1_score': f1_score(y_test, y_pred_rf_hog, average='weighted'),
        'precision': precision_score(y_test, y_pred_rf_hog, average='weighted'),
        'recall': recall_score(y_test, y_pred_rf_hog, average='weighted')
    },
    'CNN': {
        'accuracy': accuracy_score(y_test, y_pred_cnn),
        'f1_score': f1_score(y_test, y_pred_cnn, average='weighted'),
        'precision': precision_score(y_test, y_pred_cnn, average='weighted'),
        'recall': recall_score(y_test, y_pred_cnn, average='weighted')
    }
}


In [10]:
# ==================== DISPLAY TABLE ====================
table = [[model, round(metrics['accuracy'], 4), round(metrics['f1_score'], 4),
          round(metrics['precision'], 4), round(metrics['recall'], 4)]
         for model, metrics in results.items()]

print(tabulate(table, headers=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall'], tablefmt='grid'))


+-------------------+------------+------------+-------------+----------+
| Model             |   Accuracy |   F1 Score |   Precision |   Recall |
+===================+============+============+=============+==========+
| SVM               |     0.9837 |     0.9837 |      0.9837 |   0.9837 |
+-------------------+------------+------------+-------------+----------+
| SVM_HOG           |     0.9751 |     0.9751 |      0.9752 |   0.9751 |
+-------------------+------------+------------+-------------+----------+
| Random Forest     |     0.9707 |     0.9707 |      0.9707 |   0.9707 |
+-------------------+------------+------------+-------------+----------+
| Random Forest_HOG |     0.9539 |     0.9538 |      0.9539 |   0.9539 |
+-------------------+------------+------------+-------------+----------+
| CNN               |     0.9917 |     0.9917 |      0.9917 |   0.9917 |
+-------------------+------------+------------+-------------+----------+
